In [ ]:
import sys
import random
import numpy as np
from time import time
from pathlib import Path
from matplotlib import colors
import matplotlib.pyplot as plt
from IPython.display import clear_output

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import mixed_precision

from utils.plot_utils import plot_iou_trainId, plot_iou_catId
from utils.data_utils import get_labels, parse_record, get_dataset_from_tfrecord
from models.hrnet_keras import HRNet
from models.u2net import U2NET
from models.erfnet import ERFNet
from data_loader import DataLoader

K.clear_session()
physical_devices = tf.config.experimental.list_physical_devices("GPU")

def enable_amp():
    mixed_precision.set_global_policy("mixed_float16")
    
print("Tensorflow version: ", tf.__version__)
print(physical_devices,"\n")
# enable_amp() 

In [ ]:
%config InlineBackend.figure_format = 'retina'
plt.style.use('ggplot')
plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)

In [ ]:
# echo "deb [signed-by=/usr/share/keyrings/cloud.google.gpg] https://packages.cloud.google.com/apt cloud-sdk main" | sudo tee -a /etc/apt/sources.list.d/google-cloud-sdk.list
# sudo apt-get install apt-transport-https ca-certificates gnupg
# curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key --keyring /usr/share/keyrings/cloud.google.gpg add -
# sudo apt-get update && sudo apt-get install google-cloud-sdk
# gcloud init --console-only

In [ ]:
# gsutil cp gs://cl_datasets_01/cityscapes/records/trainIds_train.record /home/ubuntu/cityscapes_cv/records/
# gsutil cp gs://cl_datasets_01/cityscapes/records/trainIds_val.record /home/ubuntu/cityscapes_cv/records/
# gsutil cp gs://cl_datasets_01/cityscapes/records/trainIds_train_extra.record /home/ubuntu/cityscapes_cv/records/

In [ ]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

In [ ]:
fine = True

if fine:
    train_tfrecord_dir = "records/trainIds_train.record"
    test_tfrecord_dir = "records/trainIds_val.record"
else:
    train_tfrecord_dir = "records/trainIds_train_extra.record"

img_height = 64 # 512
img_width = 128 # 1024
n_classes = 20

labels = get_labels()
trainid2label = { label.trainId : label for label in labels }
catId2label = { label.categoryId : label for label in labels }

In [ ]:
if fine:
    TRAIN_LENGTH = 2975
    TEST_LENGTH = 500
else:
    TRAIN_LENGTH = 18000
    TEST_LENGTH = 1998

BATCH_SIZE = 6
BUFFER_SIZE = 500

In [ ]:
pipeline = DataLoader(img_height=img_height, img_width=img_width, n_classes=n_classes)

In [ ]:
if fine:
    train_ds = get_dataset_from_tfrecord(train_tfrecord_dir)
    test_ds = get_dataset_from_tfrecord(test_tfrecord_dir)
else:
    all_ds = get_dataset_from_tfrecord(train_tfrecord_dir)
    train_ds = all_ds.skip(TEST_LENGTH)
    test_ds = all_ds.take(TEST_LENGTH) 

In [ ]:
# Preprocessing: random crop the images and masks, flip them
train = train_ds.map(pipeline.load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test = test_ds.map(pipeline.load_image_test)
eval = test_ds.map(pipeline.load_image_eval)

In [ ]:
train_dataset = train.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test.batch(BATCH_SIZE)

In [ ]:
def label_to_rgb(mask):
    h = mask.shape[0]
    w = mask.shape[1]
    mask_rgb = np.zeros((h, w, 3), dtype=np.uint8)
    for val, key in trainid2label.items():
        indices = mask == val
        mask_rgb[indices.squeeze()] = key.color 
    return mask_rgb


def display(display_list, title=True):
    plt.figure(figsize=(15, 5)) # dpi=200
    if title:
        title = ['Input Image', 'True Mask', 'Predicted Mask']
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        if title:
            plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
for image, mask in train.take(21): # 16
    sample_image, sample_mask = image, mask

sample_mask = tf.argmax(sample_mask, axis=-1)
sample_mask = sample_mask[..., tf.newaxis]
sample_mask = label_to_rgb(sample_mask.numpy())
display([sample_image, sample_mask])

In [ ]:
if fine:
    MODEL_PATH = "weights/HRNet_W18.h5"
else:
    MODEL_PATH = "weights/HRNet_W18_coarse.h5"

In [ ]:
def create_mask(pred_mask):
    pred_mask = tf.squeeze(pred_mask)
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    pred_mask = label_to_rgb(pred_mask.numpy())
    return pred_mask


def show_predictions():
    pred_mask = model.predict(sample_image[tf.newaxis, ...])
    if "u2net" in model.name:
        pred_mask = pred_mask[0]
    display([sample_image, sample_mask, create_mask(pred_mask)])

In [ ]:
train_dataset = strategy.experimental_distribute_dataset(train_dataset)
test_dataset = strategy.experimental_distribute_dataset(test_dataset)

### Custom loop

In [ ]:
logger_file = "cityscapes_fine_hrnet.csv"

c_weights = [0,      0.8373, 0.918,  0.866,  1.0345, 
             1.0166, 0.9969, 0.9754, 1.0489, 0.8786,
             1.0023, 0.9539, 0.9843, 1.1116, 0.9037,
             1.0865, 1.0955, 1.0865, 1.1529, 1.0507]

In [ ]:
with strategy.scope():

    learning_rate_fn = PolynomialDecay(
        initial_learning_rate = 0.01,
        decay_steps = 12000,
        end_learning_rate=1e-5,
        power=0.9
    )

    # model = U2NET(input_height=img_height, input_width=img_width, n_classes=n_classes)
    # model = HRNet(input_height=img_height, input_width=img_width, n_classes=20, W=40)
    model = HRNet(input_height=img_height, input_width=img_width, n_classes=20, W=18)

    optimizer = SGD(learning_rate=learning_rate_fn, momentum=0.9, decay=0.0005)

In [ ]:
def iou_coef(y_true, y_pred):
    y_true = tf.nn.softmax(y_true, axis=-1)
    smooth = 1
    iou_total = 0
    for i in range(1, n_classes):
        tp = tf.math.reduce_sum(y_pred[:,:,:,i] * y_true[:,:,:,i], axis=(1,2))
        fn = tf.math.reduce_sum(y_true[:,:,:,i] * (1 - y_pred[:,:,:,i]), axis=(1,2)) 
        fp = tf.math.reduce_sum(y_pred[:,:,:,i] * (1 - y_true[:,:,:,i]), axis=(1,2)) 
        iou = tf.math.reduce_mean(tf.math.divide_no_nan(tp+smooth, tp+fn+fp+smooth), axis=0)
        iou_total += iou

    iou_macro = iou_total / (n_classes - 1)
    return iou_macro


loss_object = CategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)

def compute_loss(y_true, y_pred):
    #class_weights = tf.constant([c_weights])
    #weights_processed = tf.reduce_sum(class_weights * y_true, axis=-1)
    per_example_loss = loss_object(y_true, y_pred)
    #per_example_loss = weights_processed * per_example_loss
    return tf.nn.compute_average_loss(per_example_loss, global_batch_size=BATCH_SIZE)


def train_step(inputs):
    x, y = inputs
    with tf.GradientTape() as tape :
        output = model(x, training=True)
        loss = compute_loss(y, output)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss


@tf.function
def distributed_train_step(dist_inputs):
    per_replica_losses = strategy.run(train_step, args=(dist_inputs,))
    return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)


def test_step(inputs):
    x, y = inputs
    output = model(x, training=False)
    loss = compute_loss(y, output)
    miou = iou_coef(y, output)
    return loss, miou


@tf.function
def distributed_test_step(dist_inputs):
    pe_loss, pe_miou = strategy.run(test_step, args=(dist_inputs,))
    loss = strategy.reduce(tf.distribute.ReduceOp.SUM, pe_loss, axis=None)
    miou = strategy.reduce(tf.distribute.ReduceOp.MEAN, pe_miou, axis=None)
    return loss, miou

In [ ]:
def train(epochs):
    
    logfile = Path(logger_file)

    if not logfile.exists() :
        tmpf = logfile.open("w+")
        top_text = "epoch, loss, val_loss, miou\n"
        tmpf.write(top_text)
        tmpf.close()
    
    history = {"loss": [], "val_loss": [], "val_iou_coef": []}
    
    with strategy.scope():
    
        start_time = time()
        for epoch in range(epochs):
            epoch_start_time = time()
            
            train_losses = []
            valid_losses = []
            valid_mious = []

            for step, train_batch in enumerate(train_dataset):
                loss = distributed_train_step(train_batch)
                train_losses.append(loss.numpy())
                print("\r Batch {} -- loss: {:.4f}".format(step, loss.numpy()), end='')

            for val_batch in test_dataset:
                loss, miou = distributed_test_step(val_batch)
                valid_losses.append(loss.numpy())
                valid_mious.append(miou.numpy())

            train_loss = np.mean(train_losses)
            valid_loss = np.mean(valid_losses)
            valid_miou = np.mean(valid_mious)

            history['loss'].append(train_loss)
            history['val_loss'].append(valid_loss)
            history['val_iou_coef'].append(valid_miou)
            
            tmpf = logfile.open("a+")
            tmpf.write(",".join([str(epoch), str(train_loss), str(valid_loss), str(valid_miou)]) + "\n")
            tmpf.close()
            
            t_epoch = time() - epoch_start_time
            template = '\n Epoch {} -- Time: {:.2f}s, Loss: {:.4f}, Val Loss: {:.4f}, Val mIoU: {:.4f}'
            print(template.format(epoch+1, t_epoch, train_loss, valid_loss, valid_miou))
            
            model.save(MODEL_PATH)
            show_predictions()
            
    end_time = time()
    t_minutes = (end_time - start_time) // 60
    print("Training finished in {:.2f} minutes".format(t_minutes))
    return model, history

In [ ]:
model, history = train(epochs=300)

In [ ]:
def plot_history(history, model):
        
    plt.figure(figsize=(15,7))
    plt.subplot(1,2,1)  
    if "u2net" in model.name:
        plt.plot(history['d0_loss'], 'r', label='Training loss')
        plt.plot(history['val_d0_loss'], 'b', label='Validation loss')
    else: 
        plt.plot(history['loss'], 'r', label='Training loss')
        plt.plot(history['val_loss'], 'b', label='Validation loss')
    plt.title("Loss: "+model.name, fontsize=16)
    plt.xlabel('Epoch', fontsize=16)
    plt.legend(prop={'size': 14})

    plt.subplot(1,2,2)
    if "u2net" in model.name:
        plt.plot(history['d0_iou_coef'], 'r', label='IoU coefficient')
        plt.plot(history['val_d0_iou_coef'], 'b', label='Validation IoU coefficient')
    else:
        plt.plot(history['iou_coef'], 'r', label='IoU coefficient')
        plt.plot(history['val_iou_coef'], 'b', label='Validation IoU coefficient')
    plt.title('IoU Coefficient: '+model.name, fontsize=16)
    plt.xlabel('Epoch', fontsize=16)
    plt.legend(prop={'size': 14})
    if fine:
        plt.savefig("plots/"+model.name+"_learning_curves.png")
    else:
        plt.savefig("plots/"+model.name+"_learning_curves_coarse.png")
    plt.show()

In [ ]:
plot_history(history, model)

### Evaluation

In [ ]:
def evaluate_iou(model, dataset, n_samples):
    
    iou_scores = np.zeros((n_samples,))
    inf_times = np.zeros((n_samples, ))
    miou_op =  tf.keras.metrics.MeanIoU(num_classes=n_classes-1)
    
    for idx, (image, mask) in enumerate(dataset):
        print("\r Predicting {} \ {} ".format(idx+1, n_samples), end='')
        
        X = np.expand_dims(image.numpy(), axis=0)
        y_true = np.expand_dims(mask.numpy(), axis=0)
        
        t_start = time()
        y_pred = model.predict(X)
        t_end = time()
        t_inf = t_end-t_start
        
        inf_times[idx] = t_inf
        
        if "U2Net" in model.name:
            y_pred = y_pred[0]
            
        # y_pred = tf.image.resize(y_pred, (1024, 2048))
        threshold = tf.math.reduce_max(y_pred, axis=-1, keepdims=True)
        y_pred = tf.logical_and(y_pred >= threshold, tf.abs(y_pred) > 1e-12)
        
        y_pred = tf.cast(tf.squeeze(y_pred, axis=0), tf.int32)
        y_true = tf.cast(tf.squeeze(y_true, axis=0), tf.int32)
        
        y_true = tf.argmax(y_true[:,:,1:], axis=-1)
        y_pred = tf.argmax(y_pred[:,:,1:], axis=-1)
                
        # miou_op.reset_states()
        miou_op.update_state(y_true, y_pred)
        iou = miou_op.result().numpy()
        iou_scores[idx] = iou
        
        if idx == (n_samples-1):
            break
    
    print("Average inference time: {:.2f}s".format(np.mean(inf_times)))
            
    return iou_scores, miou_op

In [ ]:
iou_macro, miou_op = evaluate_iou(model=model, dataset=eval, n_samples=TEST_LENGTH)
iou_mean = np.mean(iou_scores)

In [ ]:
union_int = np.sum(miou_op.get_weights()[0], axis=0)+np.sum(miou_op.get_weights()[0], axis=1)
inters = np.diag(miou_op.get_weights()[0])
ious = inters / (union_int-inters+1)

print("Mean IoU: {:.4f} \n".format(iou_mean))
for i in range(ious.shape[0]) :
    print("IoU for {}: {:.2f}".format(trainid2label[i+1].name, np.round(ious[i], 2)))

In [ ]:
def plot_iou_catId(catId_label_map, n_classes, iou_class, model, iou_mean):

    categories = [catId_label_map[i].category for i in range(1, n_classes)]
    cat_colors = {
        'void': colors.to_hex(list(np.array(catId_label_map[0].color)/255)),
        'flat': colors.to_hex(list(np.array(catId_label_map[1].color)/255)),
        'construction': colors.to_hex(list(np.array(catId_label_map[2].color)/255)),
        'object': colors.to_hex(list(np.array(catId_label_map[3].color)/255)),
        'nature': colors.to_hex(list(np.array(catId_label_map[4].color)/255)),
        'sky': colors.to_hex(list(np.array(catId_label_map[5].color)/255)),
        'human': colors.to_hex(list(np.array(catId_label_map[6].color)/255)),
        'vehicle': colors.to_hex(list(np.array(catId_label_map[7].color)/255))
    }
    _colors = [cat_colors[category] for category in categories]
    
    fig, ax = plt.subplots(figsize=(14,10))
    hbars = ax.barh(categories, iou_class, color=_colors)
    
    ax.set_xlabel("IoU Coefficient: ", fontsize=18)
    ax.set_ylabel("Category Name", fontsize=18)
    ax.set_title("Category Scores for {} - Mean IoU: {:.3f}".format(model.name, iou_mean), fontsize=22)
    ax.set_xlim([0, 1])
    
    # ax.bar_label(hbars, fmt="%.2f", padding=3, fontsize=16)
    
    plt.savefig("plots/"+model.name+"_category_iou_scores.png")
    plt.show()
    
    
    
def plot_iou_trainId(trainId_label_map, catId_label_map, n_classes, iou_class, model, iou_mean):

    categories = [trainId_label_map[i].category for i in range(1, n_classes)]
    
    cat_colors = {
        'void': colors.to_hex(list(np.array(catId_label_map[0].color)/255)),
        'flat': colors.to_hex(list(np.array(catId_label_map[1].color)/255)),
        'construction': colors.to_hex(list(np.array(catId_label_map[2].color)/255)),
        'object': colors.to_hex(list(np.array(catId_label_map[3].color)/255)),
        'nature': colors.to_hex(list(np.array(catId_label_map[4].color)/255)),
        'sky': colors.to_hex(list(np.array(catId_label_map[5].color)/255)),
        'human': colors.to_hex(list(np.array(catId_label_map[6].color)/255)),
        'vehicle': colors.to_hex(list(np.array(catId_label_map[7].color)/255))
    }
    _colors = [cat_colors[category] for category in categories]

    names = [trainId_label_map[i].name for i in range(1, n_classes)]

    fig, ax = plt.subplots(figsize=(14,10))
    hbars = ax.barh(names, iou_class, color=_colors)
    
    ax.set_xlabel("IoU Coefficient: ", fontsize=18)
    ax.set_ylabel("Class Name", fontsize=18)
    ax.set_title("Class Scores for {} - Mean IoU: {:.3f}".format(model.name, iou_mean), fontsize=22)
    ax.set_xlim([0, 1])
    
    # ax.bar_label(hbars, fmt="%.2f", padding=3, fontsize=16)
    
    plt.savefig("plots/"+model.name+"_class_iou_scores.png")
    plt.show()

In [ ]:
plot_iou_trainId(
    trainId_label_map=trainid2label,
    catId_label_map=catid2label, 
    n_classes=n_classes, 
    iou_class=ious,
    model=model, 
    iou_mean=iou_mean,
)

In [ ]:
def visualize_confusion_matrix(confusion, metric, label_classes, model):
    plt.figure(figsize=(14,14))
    plt.title("{} Confusion Matrix, with Mean IoU = {:.3f}".format(model.name, metric), fontsize=22)
    plt.imshow(confusion)
    ax, fig = plt.gca(), plt.gcf()
    plt.xticks(np.arange(len(label_classes)), label_classes)
    plt.yticks(np.arange(len(label_classes)), label_classes)
    # set horizontal alignment mode (left, right or center) and rotation mode(anchor or default)
    plt.setp(ax.get_xticklabels(), rotation=-90, ha="center", rotation_mode="default")
    # avoid top and bottom part of heatmap been cut
    ax.set_xticks(np.arange(len(label_classes) + 1) - .5, minor=True)
    ax.set_yticks(np.arange(len(label_classes) + 1) - .5, minor=True)
    ax.tick_params(which="minor", bottom=False, left=False)
    ax.grid(False)
    fig.tight_layout()
    plt.show()

In [ ]:
visualize_confusion_matrix(
    confusion = miou_op.get_weights()[0] / np.sum(miou_op.get_weights()[0], axis=0), 
    metric = iou_mean_macro, 
    label_classes = [trainid2label[i].name for i in range(1, n_classes)],
    model = model
)